# Solar System Cost Analysis 

## Objective 

How much an energy system cost is a very important standard to judge if this solar storage setup is good or not? In this example, I will discover the power of mathematical optimization in addressing a common energy industry problem: solar storage setup cost analysis. 

The current ERCOT solar grid will be evaluated based on the historical Houston Hub settlement point price along with the normalized solar output in 2019.

---
## Problem Description

In this problem, a renewable energy developer want to make an investment to a new solar farm in Houston. However, they want to know what kind of configurations is most economical and beneficial for them. Therefore, an analysis on how much they would earn in 2019 by comparing three different configurations is implemented. Here, those three configurations are respectively (1) standalone storage system; (2) solar+storage system (3) standalone solar. 

The data set of 8760 hourly solar irradiance and Houston hub electricity price is as follows:

| weather year hour beginning | Solar Shape (Normalized) | Houston hub Settlement Point Energy Price (\$/MWh) |
| --- | --- | ---| 
| 1/1/2019 0:00| 0 | 14.1900	 | 
| 1/1/2019 1:00 | 0 | 15.1825	 |
| 1/1/2019 2:00 | 0 | 15.5300 | 


(1) Proposed Installation (Standalone Solar) Setting:

| Parameter | Value| Unit |
| --- | --- | --- | 
| Solar Size | 300 |  MW | 
| Inverter Capacity | 220 | MW |
| Solar System Cost | \$1.45 | \$/Watt-DC |


(2) Proposed Installation (Standalone Storage) Setting:

| Parameter | Value| Unit |
| --- | --- | --- | 
| Storage Capacity | 50 |  MW | 
| Storage Energy | 200 | MWh |
| Round Trip Efficiency | 85\% |   |
| Storage System Cost | \$350 | \$/KWh  |

(3) Proposed Installation (Combined Solar + Storage System) Setting:

| Parameter | Value| Unit |
| --- | --- | --- | 
| Solar Size | 300 |  MW | 
| Inverter Capacity | 220 | MW |
| Storage Capacity | 50 |  MW |
| Storage Energy | 200 | MWh  |
| Round Trip Efficiency | 85\% |  |
| Solar System Cost | \$1.35 | \$/Watt-DC  |
| Storage System Cost | \$325 | \$/KWh  |

---

## Model Formulation

### Parameters
$\text{RTM} \in \mathbb{R}^+$: Real Time Market Settlement Point Price  \$/KWh.

normSolar: Normalized Solar Shape.

timeStamp: hours begining in year 2019.

Life cycle of Solar Pannel: assume 30 years in service.

Life cycle of Battery Storage: assume 10 years in service.

$\eta$: Round Trip Efficiency

$SOC_t$: State of Charge at hour t

$AC_t$: AC output of solar energy at hour t

$DC_t$: DC output of solar energy at hour t

### Decision Variables

$SOC_t$: State of Charge at hour t

$E_t^{in}$: Energy delivered to the storage at hour t

$E_t^{out}$: Energy discharged to the grid at hour t

$E_t^{Gin}$ : Energy charged to the storage from grid at hour t

$E_t^{BoutG}$ : Energy discharged to the grid from storage at hour t

$E_t^{Sin}$ : Energy delivered to the storage from solar at hour t

$E_t^{SoutG}$   : Energy delivered to the grid from solar at hour t


### Objective Function

- **𝑅𝑒𝑣𝑒𝑛𝑢𝑒**: Maximize the revenue of solar system (in USD).

For a standalone solar system (Assume 30 years solar plant life):

\begin{equation}
\text{Maximize}  AC_t \times P_t
\end{equation}

For a standalone storage system (Assume 10 years storage life):

\begin{equation}
\text{Maximize}  (E_t^{out}- E_t^{in}) \times P_t
\end{equation}

For a solar and storage system (Assume 10 years storage life, 30 years solar plant life):

\begin{equation}
\text{Maximize} (E_t^{BoutG}+E_t^{SoutG})∗P_t− E_t^{Gin}∗P_t
\end{equation}

### Constraints

For a standalone solar system:

$DC_t$ = 300*normSolar

$AC_t$ = min(220, $DC_t$)

For a standalone storage system:
(1) at any time t, the SOC will follow
\begin{equation}
SOC_t=SOC_{t-1}+\sqrt{\eta}\times E_{t-1}^{in}-E_{t-1}^{out}
\end{equation}

(2) $\forall t$, $0<=SOC_t<=200$.

(3) $\forall t$, $0<=\sqrt{\eta}\times E_{t}^{in}<=50$.

(4) $\forall t$, $0<=E_{t}^{out}<=50$.

(5) $\forall t$, $0<=E_{t}^{out}<=SOC_t$.

For a system combined solar and battery storage:

(1) at any time t, the SOC will follow
\begin{equation}
SOC_t=SOC_{t-1}+ (E_{t-1}^{Gin}+E_{t-1}^{Sin})-E_{t-1}^{BoutG}
\end{equation}

(2) $\forall t$, $0<=SOC_t<=200$.

(3) $\forall t$, $0<=(E_{t}^{Gin}+E_{t}^{Sin}) <=50$.

(4)  $\forall t$, $0<=E_{t}^{BoutG}<=50$.

(5)  $\forall t$, $0<=E_{t}^{BoutG}<=SOC_t$.

---
## Python Implementation

We import the Pyomo and Gurobi Python Module and other Python libraries such as pandas, NumPy.

In [1]:
import pandas as pd
import numpy as np
import dataframe_image as dfi
import datetime
from pyomo.environ import *
import warnings
warnings.filterwarnings('ignore')

## Input Data
We define all the input data of the model.

In [ ]:
file_loc = "./case/case/price/pricee.xlsx"

dti = pd.date_range("2019-01-01", periods=8760, freq="H")

sheets_dict = pd.read_excel(file_loc, sheet_name=None)

dflist=[]
for i in sheets_dict.values():
    dfnewprice = i.loc[i['Settlement Point Name'] == 'HB_HOUSTON']
    df3 = dfnewprice['Settlement Point Price']
    df3.index = range(len(df3.index))
    df6 = df3.groupby(df3.index // 4).mean()
    dflist.append(df6)

dfhourprice = pd.concat(dflist)
dfhourprice.index = dti  
dfhourprice = dfhourprice.to_frame()

# solar shape
file_loc = "./case/case/FuelMixReport_PreviousYears/IntGenbyFuel2019.xlsx"
sheetname= ['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec']

dfqqlist=[]

for i in sheetname:
    df_price_map = pd.read_excel(file_loc, sheet_name=i)
    dfnewprice = df_price_map.loc[df_price_map['Fuel'] == 'Solar']
    dfnewprice1 = dfnewprice.iloc[: , 4:]
    dfnewprice1 = dfnewprice1.groupby(np.arange(len(dfnewprice1.columns))//4, axis=1).mean()
    dfnewprice1.index = range(len(dfnewprice1.index))
    dfnewprice1_transposed = dfnewprice1.T # or df1.transpose(
    dfnewprice1_transposed1 = pd.Series(dfnewprice1_transposed.values.ravel('F'))
    dfqqlist.append(dfnewprice1_transposed1)

dfnormsolar = pd.concat(dfqqlist)

normalized_dfnormsolar=dfnormsolar/dfnormsolar.max()
normalized_dfnormsolar.index = dti

df = dfhourprice.merge(normalized_dfnormsolar.rename('SolarShape'), left_index=True, right_index=True)

## Compute the monthly averaged historical  Settlement Point Price for the year of 2019

In [ ]:
monthly_averages = df.groupby(pd.PeriodIndex(df.index, freq="M"))['Settlement Point Price'].mean()
hourly_averages = df.groupby([df.index.month, df.index.hour])['Settlement Point Price'].mean()
dfavg1 = pd.DataFrame()

for i in range (1,13):
    for j in range (24):
        dftemp=df.loc[(df.index.month==i) & (df.index.hour==j), 'Settlement Point Price'] 
        dfavg1.loc[i-1,j+1] = round(dftemp.mean())
dfavg1.index = ['01', '02', '03', '04','05','06','07','08','09','10','11','12']
dfavg1.loc['Avg'] = round(dfavg1.mean(axis=0))

dfavg1['Avg'] = round(dfavg1.mean(axis=1))
dfavg1 = dfavg1.astype(int)
df_styled = dfavg1.style.background_gradient(axis=None, vmin=10, vmax=50, cmap='coolwarm')
dfi.export(df_styled,"Monthly_Average_Price.png")
dfavg1.to_excel("avgRTM.xlsx")
df_styled

In [ ]:
sp1={'Solar Size': 300.0, 'Inverter Capacity': 220.0, 'Solar System Cost': 1.45}

sp2={'Storage Capacity': 50,
 'Storage Energy': 200,
 'Round Trip Efficiency': 0.85,
 'Storage System Cost': 350}

sp3={'Solar Size': 300.0,
 'Inverter Capacity': 220.0,
 'Storage Capacity': 50.0,
 'Storage Energy': 200.0,
 'Round Trip Efficiency': 0.85,
 'Solar System Cost': 1.35,
 'Storage System Cost': 325}

### case 1: Calculate the revenue of Standalone Solar configuration

In [ ]:
df['SolarDC'] = df['SolarShape'].values * sp1['Solar Size']
df['SolarAC'] = df['SolarDC'].apply(lambda x: x if x <= sp1['Inverter Capacity'] else sp1['Inverter Capacity'])
df['hour'] = range(0,8760)
df.index = df.hour
results_df = df 
results_df['revenue'] = df.SolarAC * df['Settlement Point Price'] 
serviceLifeSolar = 30

total_revenue = results_df.revenue.sum()
capex = sp1['Solar Size']*sp1['Solar System Cost']*1000000/serviceLifeSolar

print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual cost ${:,.2f}'.format(capex))

### Case 2: Calculate the revenue of Standalone Storage configuration

In [ ]:
def optimizeStorage(df, first_model_hour=0, last_model_hour=8759, setup=sp2):
    df = df.loc[first_model_hour:last_model_hour, :]

    model = ConcreteModel()

    # Define model parameters
    model.T = Set(doc='hour of year', initialize=df.hour.tolist(), ordered=True)
    model.Rmax = Param(initialize=setup['Storage Capacity'],
                       doc='Max rate of power flow (MW) in or out')
    model.Smax = Param(initialize=setup['Storage Energy'], doc='Max storage (MWh)')
    model.P = Param(initialize=df['Settlement Point Price'].tolist(), doc='RTMP for each hour')

    # Charge, discharge, and state of charge
    model.Ein = Var(model.T, domain=NonNegativeReals)
    model.Eout = Var(model.T, domain=NonNegativeReals)
    model.S = Var(model.T, bounds=(0, model.Smax))


    #Set all constraints
    def storage_state(model, t):
        'Storage changes with flows in/out and efficiency losses'
        if t == model.T.first():
            return model.S[t] == 0

        else:
            return (model.S[t] == (model.S[t-1] 
                                + (model.Ein[t-1] * np.sqrt(eta)) 
                                - (model.Eout[t-1])))

    model.charge_state = Constraint(model.T, rule=storage_state)

    def discharge_constraint(model, t):
        "Maximum dischage within a single hour"
        return model.Eout[t] <= model.Rmax

    model.discharge = Constraint(model.T, rule=discharge_constraint)

    def charge_constraint(model, t):
        "Maximum charge within a single hour"
        return model.Ein[t]*np.sqrt(eta) <= model.Rmax

    model.charge = Constraint(model.T, rule=charge_constraint)

    def positive_charge(model, t):
        'Limit discharge to the amount of charge in battery, including losses'
        return model.Eout[t] <= model.S[t] 
    model.positive_charge = Constraint(model.T, rule=positive_charge)

    # Define the battery income, expenses, and profit
    income = sum(df.loc[t, 'Settlement Point Price' ] * model.Eout[t]*np.sqrt(eta) for t in model.T)
    expenses = sum(df.loc[t, 'Settlement Point Price'] * model.Ein[t] for t in model.T)
    profit = income - expenses
    model.objective = Objective(expr=profit, sense=maximize)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model)

    hours = range(model.T[first_model_hour + 1], model.T[last_model_hour + 1] + 1)
    Ein = [value(model.Ein[i]) for i in hours]
    Eout = [value(model.Eout[i]) for i in hours]
    Price = [model.P.extract_values()[None][i] for i in hours]
    charge_state = [value(model.S[i]) for i in hours]

    df_dict = dict(
        hour=hours,
        Ein=Ein,
        Eout=Eout,
        price=Price,
        charge_state=charge_state
    )

    results_df = pd.DataFrame(df_dict)

    results_df['timeStamp'] = df.index

    return results_df

eta = 0.85 # Round trip storage efficiency
results_df = optimizeStorage(df,first_model_hour=0, last_model_hour=8759, setup = sp2)
results_df.to_csv('optimization_resultscase2.csv')


results_df['revenue'] = results_df.Eout*np.sqrt(eta)* results_df.price - results_df.Ein * results_df.price 
total_revenue = results_df.revenue.sum()

serviceLife = 10
capex = sp2['Storage System Cost']*sp2['Storage Energy']*1000/serviceLife

print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual cost ${:,.2f}'.format(capex))

### Case 3: Calculate the projected backcast 2019 revenue of Solar + Storage configuration
Here, two settings are considered: (1) AC coupled (left), (2) DC coupled (right).

AC coupled  | DC coupled
- | - 
![alt](./Picture1.png) | ![alt](./Picture2.png)

Additional Constraints:

For AC coupled:

$\forall t$, $0<=E_t^{SoutG}+E_t^{Sin}/\sqrt{\eta}<=AC_t$


For DC coupled:

$\forall t$, $0<=E_t^{SoutG}+E_t^{Sin}/\sqrt{\eta}<=DC_t$

$\forall t$, $0<=E_t^{SoutG}+E_t^{BoutG}*\sqrt{\eta}<=220$

In [ ]:
def optimizeSolStorage(df, first_model_hour=0, last_model_hour=8759, setup=sp3):
    df = df.loc[first_model_hour:last_model_hour, :]

    model = ConcreteModel()

    # Define model parameters
    model.T = Set(doc='hour of year', initialize=df.hour.tolist(), ordered=True)

    model.Rmax = Param(initialize=setup['Storage Capacity'],
                       doc='Max rate of power flow (MW) in or out')
    model.Smax = Param(initialize=setup['Storage Energy'], doc='Max storage (MWh)')
    model.P = Param(initialize=df['Settlement Point Price'].tolist(), doc='RTMP for each hour')
    eta = 0.85 # Round trip storage efficiency

    # Charge in from grid, discharge out to grid, and state of charge
    model.GinE = Var(model.T, domain=NonNegativeReals)
    model.EoutG = Var(model.T, domain=NonNegativeReals)
    model.S = Var(model.T, bounds=(0, model.Smax))


    # Solar output to grid
    # Solar output to storage
    model.SinE = Var(model.T, domain=NonNegativeReals)
    model.SoutG = Var(model.T, domain=NonNegativeReals)

    model.ACout = Param(initialize=df.SolarAC.tolist(),doc='Solar AC')

    #Set all constraints
    def storage_state(model, t):
        'Storage changes with flows in/out and efficiency losses'
        if t == model.T.first():
            return model.S[t] == 0

        else:
            return (model.S[t] == (model.S[t-1] 
                                + (model.GinE[t-1] ) 
                                + (model.SinE[t-1] ) 
                                - (model.EoutG[t-1])))

    model.charge_state = Constraint(model.T, rule=storage_state)

    def discharge_constraint(model, t):
        "Maximum dischage within a single hour"
        return model.EoutG[t] <= model.Rmax

    model.discharge = Constraint(model.T, rule=discharge_constraint)

    def charge_constraint(model, t):
        "Maximum charge within a single hour"
        return (model.GinE[t]+model.SinE[t]) <= model.Rmax

    model.charge = Constraint(model.T, rule=charge_constraint)

    def positive_charge(model, t):
        'Limit discharge to the amount of charge in battery, including losses'
        return model.EoutG[t] <= model.S[t] 
    model.positive_charge = Constraint(model.T, rule=positive_charge)


    def inverter_limit(model, t):
        'Limit inverter output'
        return model.SoutG[t] + model.SinE[t]/np.sqrt(eta)<=model.ACout.extract_values()[None][t]
 
    model.inverter_limit = Constraint(model.T, rule=inverter_limit)



    # Define the total income, expenses, and profit
    income = sum(df.loc[t, 'Settlement Point Price'] * (model.EoutG[t]*np.sqrt(eta) + model.SoutG[t]) for t in model.T)
    expenses = sum(df.loc[t, 'Settlement Point Price'] * model.GinE[t]/np.sqrt(eta) for t in model.T)
    profit = income - expenses
    model.objective = Objective(expr=profit, sense=maximize)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model)

    hours = range(model.T[first_model_hour + 1], model.T[last_model_hour + 1] + 1)
    GinE = [value(model.GinE[i]) for i in hours]
    EoutG = [value(model.EoutG[i]) for i in hours]
    SinE = [value(model.SinE[i]) for i in hours]
    SoutG = [value(model.SoutG[i]) for i in hours]    
    Price = [model.P.extract_values()[None][i] for i in hours]
    charge_state = [value(model.S[i]) for i in hours]

    df_dict = dict(
        hour = hours,
        GinE = GinE,
        EoutG = EoutG,
        SinE = SinE,
        SoutG = SoutG,
        price = Price,
        charge_state = charge_state
    )

    results_df = pd.DataFrame(df_dict)

    results_df['timeStamp'] = df.index

    return results_df

results_df = optimizeSolStorage(df,first_model_hour=0, last_model_hour=8759, setup = sp3)
results_df.to_csv('optimization_resultscase3AC.csv')


results_df['revenue'] = (results_df.EoutG*np.sqrt(eta)+ results_df.SoutG)* results_df.price - (results_df.GinE/np.sqrt(eta)) * results_df.price
total_revenue = results_df.revenue.sum()

serviceLifeStorange = 10
serviceLifeSolar = 30

capex = sp3['Storage System Cost']*sp3['Storage Energy']*1000/serviceLifeStorange + sp3['Solar Size']*sp3['Solar System Cost']*1000000/serviceLifeSolar


print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual cost ${:,.2f}'.format(capex))

In [ ]:
def optimizeSolStorage(df, first_model_hour=0, last_model_hour=8759, setup=sp3):
    df = df.loc[first_model_hour:last_model_hour, :]

    model = ConcreteModel()

    # Define model parameters
    model.T = Set(doc='hour of year', initialize=df.hour.tolist(), ordered=True)

    model.Rmax = Param(initialize=setup['Storage Capacity'],
                       doc='Max rate of power flow (MW) in or out')
    model.Smax = Param(initialize=setup['Storage Energy'], doc='Max storage (MWh)')
    model.P = Param(initialize=df['Settlement Point Price'].tolist(), doc='RTMP for each hour')
    eta = 0.85 # Round trip storage efficiency

    # Charge in from grid, discharge out to grid, and state of charge
    model.GinE = Var(model.T, domain=NonNegativeReals)
    model.EoutG = Var(model.T, domain=NonNegativeReals)
    model.S = Var(model.T, bounds=(0, model.Smax))


    # Solar output to grid
    # Solar output to storage
    model.SinE = Var(model.T, domain=NonNegativeReals)
    model.SoutG = Var(model.T, domain=NonNegativeReals)

    model.DCout = Param(initialize=df.SolarDC.tolist(),doc='Solar DC')

    #Set all constraints
    def storage_state(model, t):
        'Storage changes with flows in/out and efficiency losses'
        if t == model.T.first():
            return model.S[t] == 0

        else:
            return (model.S[t] == (model.S[t-1] 
                                + (model.GinE[t-1] ) 
                                + (model.SinE[t-1] ) 
                                - (model.EoutG[t-1])))

    model.charge_state = Constraint(model.T, rule=storage_state)

    def discharge_constraint(model, t):
        "Maximum dischage within a single hour"
        return model.EoutG[t] <= model.Rmax

    model.discharge = Constraint(model.T, rule=discharge_constraint)

    def charge_constraint(model, t):
        "Maximum charge within a single hour"
        return  (model.GinE[t]+model.SinE[t]) <= model.Rmax

    model.charge = Constraint(model.T, rule=charge_constraint)

    
    def positive_charge(model, t):
        'Limit discharge to the amount of charge in battery, including losses'
        return model.EoutG[t] <= model.S[t] 
    model.positive_charge = Constraint(model.T, rule=positive_charge)

    def DC_limit(model, t):
        'Limit DC output'
        return (model.SoutG[t] + model.SinE[t]/np.sqrt(eta))<=model.DCout.extract_values()[None][t]

    model.DC_limit = Constraint(model.T, rule=DC_limit)

    def inverter_limit(model, t):
        'Limit inverter output'
        return model.SoutG[t] + model.EoutG[t]*np.sqrt(eta)<=setup['Inverter Capacity']

    model.inverter_limit = Constraint(model.T, rule=inverter_limit)

    # Define the total income, expenses, and profit
    income = sum(df.loc[t, 'Settlement Point Price'] * (model.EoutG[t]*np.sqrt(eta)+ model.SoutG[t]) for t in model.T)
    expenses = sum(df.loc[t, 'Settlement Point Price'] * model.GinE[t]/np.sqrt(eta) for t in model.T)
    profit = income - expenses
    model.objective = Objective(expr=profit, sense=maximize)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model)

    hours = range(model.T[first_model_hour + 1], model.T[last_model_hour + 1] + 1)
    GinE = [value(model.GinE[i]) for i in hours]
    EoutG = [value(model.EoutG[i]) for i in hours]
    SinE = [value(model.SinE[i]) for i in hours]
    SoutG = [value(model.SoutG[i]) for i in hours]    
    Price = [model.P.extract_values()[None][i] for i in hours]
    charge_state = [value(model.S[i]) for i in hours]

    df_dict = dict(
        hour = hours,
        GinE = GinE,
        EoutG = EoutG,
        SinE = SinE,
        SoutG = SoutG,
        price = Price,
        charge_state = charge_state
    )

    results_df = pd.DataFrame(df_dict)

    results_df['timeStamp'] = dti

    return results_df

results_df = optimizeSolStorage(df,first_model_hour=0, last_model_hour=8759, setup = sp3)

results_df.to_csv('optimization_resultscase3DC.csv')


results_df['revenue'] = (results_df.EoutG*np.sqrt(eta)+ results_df.SoutG)* results_df.price - (results_df.GinE/np.sqrt(eta)) * results_df.price  
total_revenue = results_df.revenue.sum()

serviceLifeStorange = 10
serviceLifeSolar = 30

capex = sp3['Storage System Cost']*sp3['Storage Energy']*1000/serviceLifeStorange + sp3['Solar Size']*sp3['Solar System Cost']*1000000/serviceLifeSolar


print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual cost ${:,.2f}'.format(capex))



##  Using Gurobipy

(1) for the standalone storage system, it is an optimization problem (energy arbitrage).

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Parameters

nhours = 8760
StorageCapacity = 50
StorageEnergy = 200
rtm = df['Settlement Point Price'].tolist()
timeh = range(nhours)

In [ ]:
eta = 0.85
# Create a new model
m = gp.Model ("solarPlan")
# Create variables
eOut = m.addVars(timeh,  vtype = GRB.CONTINUOUS, name ="eout")
eIn = m.addVars (timeh,  vtype = GRB.CONTINUOUS , name ="ein")
soc = m.addVars (timeh, ub=StorageEnergy, vtype = GRB.CONTINUOUS, name ="soc")

## Set constraints

In [ ]:
start_state = m.addConstrs((soc[t] == 0 for t in timeh  if t==0),'SOCst')
storage_state = m.addConstrs(( soc[t] == (soc[t-1] 
                                + (eIn[t-1] * np.sqrt(eta)) 
                                - (eOut[t-1])) for t in timeh[1:]),'SOC')

In [ ]:
discharge_state = m.addConstrs((eOut[t] <= StorageCapacity for t in timeh),'discharge_st')

charge_state = m.addConstrs( (eIn[t]* np.sqrt(eta) <= StorageCapacity for t in timeh),'charge_st')

positive_charge = m.addConstrs( (eOut[t] <= soc[t] for t in timeh),'poscharge')

In [ ]:
### Define the battery income, expenses, and profit
# Set objective
m.setObjective(sum(rtm[t]*(eOut[t]* np.sqrt(eta) - eIn[t]) for t in timeh), GRB.MAXIMIZE)

In [ ]:
m.optimize()

### Case 3 AC

In [ ]:
m = gp.Model ("solarPlanAC")

In [ ]:
# add additional Parameters
# solarCost = 1.35
# batteryCost = 350
ACout =  df.SolarAC.tolist()

In [ ]:
# Create variables
GInB = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS , name ="GinB")
SinB = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="SinB")
BoutG = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="BoutG")
SoutG = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="SoutG")
soc = m.addVars (timeh, lb=0, ub=StorageEnergy, vtype = GRB.CONTINUOUS, name ="soc")

### constraints

In [ ]:
start_state = m.addConstrs((soc[t] == 0 for t in timeh  if t==0),'SOCst')
storage_state = m.addConstrs(( soc[t] == (soc[t-1] 
                                + (GInB[t-1]+SinB[t-1])
                                - (BoutG[t-1])) for t in timeh[1:]),'SOC')

In [ ]:
discharge_state = m.addConstrs((BoutG[t] <= StorageCapacity for t in timeh),'discharge_st')

charge_state = m.addConstrs( ((GInB[t]+SinB[t]) <= StorageCapacity for t in timeh),'charge_st')

positive_charge = m.addConstrs((BoutG[t] <= soc[t] for t in timeh),'poscharge')

AC_lim = m.addConstrs((SoutG[t]+SinB[t]/np.sqrt(eta) <= ACout[t]  for t in timeh),'poscharge')

In [ ]:
### Define the battery income, expenses, and profit
# Set objective
m.setObjective(sum(rtm[t]*(BoutG[t]*np.sqrt(eta)+ SoutG[t]-GInB[t]/np.sqrt(eta)) for t in timeh), GRB.MAXIMIZE)
m.optimize()

### Case 3 DC

In [ ]:
m = gp.Model ("solarPlanDC")
# add additional Parameters
DCout =  df.SolarDC.tolist()
InverterCapacity = 220

In [ ]:
# Create variables
GInB = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS , name ="GinB")
SinB = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="SinB")
BoutG = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="BoutG")
SoutG = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="SoutG")
soc = m.addVars (timeh, lb=0, ub=StorageEnergy, vtype = GRB.CONTINUOUS, name ="soc")

In [ ]:
start_state = m.addConstrs((soc[t] == 0 for t in timeh  if t==0),'SOCst')
storage_state = m.addConstrs(( soc[t] == (soc[t-1] 
                                + (GInB[t-1]+SinB[t-1])
                                - (BoutG[t-1])) for t in timeh[1:]),'SOC')

In [ ]:
discharge_state = m.addConstrs((BoutG[t] <= StorageCapacity for t in timeh),'discharge_st')

charge_state = m.addConstrs( ((GInB[t]+SinB[t])<= StorageCapacity for t in timeh),'charge_st')

positive_charge = m.addConstrs((BoutG[t] <= soc[t] for t in timeh),'poscharge')

dc_lim = m.addConstrs((SoutG[t]+SinB[t]/np.sqrt(eta) <= DCout[t]  for t in timeh),'poscharge')

inverter_lim =  m.addConstrs((SoutG[t]+BoutG[t]*np.sqrt(eta) <= InverterCapacity  for t in timeh),'poscharge')

In [ ]:
### Define the battery income, expenses, and profit
# Set objective
m.setObjective(sum(rtm[t]*(BoutG[t]*np.sqrt(eta)+ SoutG[t]-GInB[t]/np.sqrt(eta)) for t in timeh), GRB.MAXIMIZE)
m.optimize()

## OPTIMIZING ELECTRICITY TRANSMISSION COST

In [ ]:
from gurobipy import *
#sets
plants=['Plant A','Plant B','Plant C' ]
cities =['NewYork','Boston','Philadelphia','NewJersy']
I = range(len(plants))
J = range(len(cities))

#data
Costs= [[8,12,7,3],
    [4,18,6,9],
    [14,2,10,15]]

Demand = [50,30,40,20]
Supply = [35,60,50]


m = Model('Trans Model')

X={}
for i in I:
    for j in J:
        X[i,j]=m.addVar()
        

m.setObjective(quicksum(Costs[i][j]*X[i,j] for i in I for j in J), GRB.MINIMIZE)

for j in J:
    m.addConstr(quicksum(X[i,j] for i in I) >= Demand[j])

for i in I:
    m.addConstr(quicksum(X[i,j] for j in J) <= Supply[i])

m.optimize()
print('Optimized Cost', m.objVal)

# Sensitivity Analysis
for j in J:
    print('-'*20)
    print("cities:",j,"Demand:",Demand[j])
    print("-"*20)
    for i in I:
        print("     plants", i, X[i,j].x)


|Plant| NewYork | Boston| Philadelphia| NewJersy| Supply (million kwh)|
|---| --- | --- | --- | ---| --- |
|A| 0  Kwh | 0 mKwh | 15 mKwh | 20 mKwh | 35 mKwh |
|B| 50 mKwh | 0 mKwh |  10 mKwh |  0 mKwh | 60 mKwh |
|C| 0 mKwh | 30 mKwh | 15 mKwh |  0 mKwh | 45 mKwh |
|Demand (million kwh)| 50 mKwh | 30 mKwh | 40 mKwh | 20 mKwh| 

The optimum cost of transmission is \$635.